In [2]:
!pip install 'transformers[torch]' 
!pip install datasets 
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.8 MB/s eta 0:00:00


In [70]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.optim import AdamW

import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from datasets import load_dataset
from transformers import AutoTokenizer, BertForSequenceClassification, AlbertConfig, Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoModel
from transformers import default_data_collator

import evaluate

from torch.utils.data import DataLoader, TensorDataset

from tqdm import tqdm

from IPython.display import clear_output

from sklearn.model_selection import train_test_split

from sklearn.metrics.pairwise import cosine_similarity

from torch.utils.data import Dataset, DataLoader

import pandas as pd
import json
from datetime import datetime
import re

from imblearn.over_sampling import SMOTE
import datetime

In [90]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [108]:
class SeqSimModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = AutoModel.from_pretrained('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
        self.embeding = AutoModel.from_pretrained('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

    def mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0]
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        
    def forward(self, input_ids, attention_mask):
        model_output = self.model(input_ids=input_ids, attention_mask=attention_mask)
        sentence_embeddings = self.mean_pooling(model_output, attention_mask)
        return (F.cosine_similarity(sentence_embeddings[0].unsqueeze(0), sentence_embeddings[1].unsqueeze(0)) + 1) / 2

In [5]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

tokenizer_config.json:   0%|          | 0.00/402 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [109]:
num_epochs = 3

model = SeqSimModel().to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=5e-5)
scheduler = None

In [130]:
def get_proba(emp, vac, targ):
    emp_tokens = tokenizer([emp, vac], padding=True, truncation=True, return_tensors='pt')
    input_ids = emp_tokens['input_ids'].unsqueeze(0).to(device)
    attention_mask = emp_tokens['attention_mask'].unsqueeze(0).to(device)
    t = torch.tensor([targ]).unsqueeze(0).to(device)
    with torch.no_grad():
        out_proba = model(input_ids.reshape((2, -1)), attention_mask.reshape((2, -1)))
    return out_proba.item()


result = []

model.eval()
for skills, exp, about, vac, targ in tqdm(zip(list(df['key_skills']), list(df['works_descriptions_arr']), list(df['about']), list(df['vec_description']), list(df['target'])), desc=f'Test'):
    curr_result = 0
    
    if skills is not None:
        curr_result -= get_proba(skills, vac, targ)
        
    if about is not None:
        curr_result += get_proba(about, vac, targ)
        
    if exp is not None:
        for curr_exp in exp:
            curr_result += get_proba(curr_exp, vac, targ)
        
    result.append(curr_result)
    
print(len(result))

Test: 656it [03:12,  3.41it/s]

656


In [131]:
df['transformer_proba'] = result
df

,about,birth_date,city,country,first_name,key_skills,languageItems,last_name,edu_num,work_cnt,is_working,mean_work_dur,works_descriptions,works_descriptions_arr,candidate_description,vec_description,vac_uuid,target,transformer_proba
0,None,1979-01-01,Санкт-Петербург,0,Данила,"Java, Spring Boot, Java EE, SQL, Hibernate, Gi...",None,Прохоров,1,5,0,740.800000,Интеграционные адаптеры для передачи заявок ...,[ Интеграционные адаптеры для передачи заявок ...,санкт петербург java spring boot java ee sql ...,450 на руки описание мы расширяем команды и ищ...,779f3a59-206a-3241-adc4-d7db504f960b,0,2.876713
1,"(Гражданин Республики Казахстан, по России им...",1991-01-01,Москва,0,Савва,"ООП, Java, Java Spring Framework, Функциональн...",None,Исаев,1,5,1,456.250000,Разработка программного комплекса кредитного...,[ Разработка программного комплекса кредитного...,гражданин республики казахстан по россии имее...,450 на руки описание мы расширяем команды и ищ...,779f3a59-206a-3241-adc4-d7db504f960b,0,3.370661
2,"С 1999 года, после появления дома первого ком...",1986-12-08,,0,Николай,"Java, Spring Framework, Hibernate ORM, SQL, Ja...",[Английский],Новиков,3,2,1,670.000000,"• Разрабатывал backend часть по открытию, за...","[ • Разрабатывал backend часть по открытию, за...",с 1999 года после появления дома первого комп...,450 на руки описание мы расширяем команды и ищ...,779f3a59-206a-3241-adc4-d7db504f960b,0,1.444150
3,"- Oracle Certified Associate, Java SE 7 Progra...",1985-10-13,Новосибирск,0,Гавриил,"Java, Git, SQL, HTML, JavaScript, CSS, MySQL, ...","[Русский, Английский]",Новикова,2,1,1,0.000000,Full Architecture & Design & Release & Suppor...,[Full Architecture & Design & Release & Suppor...,oracle certified associate java se 7 programm...,450 на руки описание мы расширяем команды и ищ...,779f3a59-206a-3241-adc4-d7db504f960b,0,0.813582
4,None,1992-07-28,Тюмень,0,Ярослава,"Java, Spring, Git, PostgreSQL, Hibernate ORM, ...","[Русский, Английский]",Тихонова,1,2,1,701.000000,Разработка сервиса с нуля. Spring boot + pos...,[ Разработка сервиса с нуля. Spring boot + pos...,тюмень java spring git postgresql hibernate o...,450 на руки описание мы расширяем команды и ищ...,779f3a59-206a-3241-adc4-d7db504f960b,0,0.884897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651,None,1995-01-01,Тула,0,Альбина,"JavaScript, TypeScript, Vue.js, Nuxt.js, Node....",None,Мясникова,1,3,1,562.500000,Основная часть разработки была передана на а...,[ Основная часть разработки была передана на а...,тула javascript typescript vue js nuxt js nod...,бренд одежды и аксессуаров созданный в 2008 г...,259bf318-e6a7-3b6c-93f9-e1804a89ee63,1,1.725539
652,None,1988-11-14,Пенза,0,Аким,"JavaScript, TypeScript, Vue js, Nuxt, HTML, CS...",None,Семёнова,1,4,1,711.666667,"Разработка UI Stack: Vu3/2, Typescript, Nuxt,...","[Разработка UI Stack: Vu3/2, Typescript, Nuxt,...",пенза javascript typescript vue js nuxt html ...,бренд одежды и аксессуаров созданный в 2008 г...,259bf318-e6a7-3b6c-93f9-e1804a89ee63,1,2.218990
653,None,1992-06-02,Тула,0,Матвей,"JavaScript, TypeScript, Vue.js, Nuxt, Nest, No...",None,Константинов,1,4,1,588.333333,Обязанности: - разработка фронтенда на Vue 3...,[ Обязанности: - разработка фронтенда на Vue 3...,тула javascript typescript vue js nuxt nest n...,бренд одежды и аксессуаров созданный в 2008 г...,259bf318-e6a7-3b6c-93f9-e1804a89ee63,1,2.344426
654,"Профессиональные навыки: • JavaScript, базовы...",2000-01-01,Санкт-Петербург,0,Руслан,"JavaScript, TypeScript, Vue.js, Nuxt.js, Автом...",None,Нестеров,1,4,1,396.000000,Чем занимаюсь: – ввожу архитектурные подходы...,[ Чем занимаюсь: – ввожу архитектурные подходы...,профессиональные навыки javascript базовый ty...,бренд одежды и аксессуаров созданный в 2008 г...,259bf318-e6a7-3b6c-93f9-e1804a89ee63,1,3.001757


In [133]:
df.to_csv('df_with_trans_2.csv', index=False)

In [48]:
import gc
torch.cuda.empty_cache()
gc.collect()

0